In [ ]:
"""Functions for plugging into Pipulate-frameworks for conducting SEO investigations."""

In [ ]:
import requests, re

In [ ]:
test_url = 'http://mikelev.in/'

In [ ]:
def Title(**row_dict):
    a_pattern = r'<title\s?>(.*?)</title\s?>'
    a_string = row_dict['response'].text
    matches = re.findall(pattern=a_pattern, string=a_string, flags=re.M|re.I)
    if matches:
        return(200, matches[0])
    else:
        return(200, None)

In [ ]:
if __name__ == '__main__':
    print("Title: %s" % Title(URL=test_url)[1])